## Ingest lap times folder

1. Read the JSON file using the spark dataframe reader

In [0]:
from pyspark.sql.functions import col, lit
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
lap_times_schema = StructType(
    fields=[
        StructField("raceId", IntegerType(), False),
        StructField("driverId", IntegerType(), True),
        StructField("lap", IntegerType(), True),
        StructField("position", IntegerType(), True),
        StructField("time", StringType(), True),
        StructField("milliseconds", IntegerType(), True)
    ]
)

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
lap_times_df = spark.read \
  .schema(lap_times_schema) \
  .csv(f'{raw_folder_path}/lap_times')
# display(lap_times_df)
# lap_times_df.count()

2. Process Dataframe

In [0]:
lap_times_final_df = lap_times_df.withColumnRenamed("raceId", "race_id") \
    .withColumnRenamed("DriverId", "driver_id") \
    .withColumn("data_source", lit(v_data_source))
lap_times_final_df = add_ingestion_date(lap_times_final_df)
#display(lap_times_final_df)

3. Write data to datalake as parquet

In [0]:
#lap_times_final_df.write.mode("overwrite").parquet(f"{processed_folder_path}/lap_times")
# database
lap_times_final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_processed.lap_times")

In [0]:
# %fs
# ls mnt/formula1jf/processed/lap_times

In [0]:
dbutils.notebook.exit("Success")